Author: <font color="blue">Alan Le Roux Osorio</font>

E-mail: <font color="blue">alan.leroux01@estudiant.upf.edu</font>

Date: <font color="blue">26/11/2025</font>

In [1]:
import io
import nltk
import gzip
import random
import statistics
import secrets
import re
import gzip

# 0. Dataset and how to iterate

In [2]:
# LEAVE THIS CODE AS-IS

INPUT_FILE = "movie_lines.tsv.gz"

In [3]:
# LEAVE THIS CODE AS-IS

POS_NOUN = 'NN'
POS_VERB = 'VB'
POS_ADJECTIVE = 'JJ'

# Producer in Python that reads a file by words that are nouns
def read_by_parts_of_speech(filename, parts_of_speech, max_words=-1, report_every=-1):
    
    # Default value for report_every
    if report_every < 0 and max_words > 0:
        report_every = int(max_words/5)
    
    # Open the input file
    with gzip.open(INPUT_FILE, "rt", encoding='utf8') as file:
        
        # Initialize counter of words to stop at max_words
        counter = 0
    
        # Iterate through lines in the file
        for line in file:
            
            elements = line.split("\t")
            
            text = ""
            if len(elements) >= 5:
                text = elements[4].strip()
                                        
            if counter > max_words and max_words != -1:
                break
                
            for sentence in nltk.sent_tokenize(text):
                
                tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
                for word in [part[0] for part in tagged if part[1] in parts_of_speech]:
                
                    counter += 1

                    # Report
                    if (report_every != -1) and (counter % report_every == 0):
                        if max_words == -1:
                            print("- Read %d words so far" % (counter))
                        else:
                            print("- Read %d/%d words so far" % (counter, max_words))

                    # Produce the word in lowercase
                    yield word.lower()

In [4]:
import nltk
#ChatGPT recommended these downloads to ensure no errors as I couldn't find why it didn't work
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')



[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\lerou\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lerou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lerou\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\lerou\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [5]:
# MODIFY THE max_words PARAMETER TO TAKE ABOUT 30 SECONDS TO DO THIS

# Maximum number of words to read, one pass should take about 30 seconds in your computer
# 8000 wordstake 31.8 in my computer
#30000 words took 2 minutes and 39 seconds in my computer
MAX_WORDS = 30000

for word in read_by_parts_of_speech(INPUT_FILE, [POS_ADJECTIVE], max_words=MAX_WORDS):
    # Prints 1/1000 of words
    if random.random() < 0.001:
        print("Current noun '%s'" % (word)) 

Current noun 'able'
Current noun 'true'
Current noun 'child'
Current noun 'sure'
Current noun 'beautiful'
Current noun 'long'
Current noun 's'posed'
Current noun 'white'
Current noun 'groovy'
Current noun 'small'
- Read 6000/30000 words so far
Current noun 'right'
Current noun 'ta'
Current noun 'sharp'
Current noun 'fine'
Current noun 'good'
- Read 12000/30000 words so far
Current noun 'last'
Current noun 'last'
Current noun 'hidden'
Current noun 'secret'
Current noun 'beautiful'
Current noun 'new'
Current noun 'curious'
- Read 18000/30000 words so far
Current noun 'evil'
Current noun 'hard'
Current noun 'due'
Current noun 'subconscious'
Current noun 'fifty'
Current noun 'cosmic'
- Read 24000/30000 words so far
Current noun 'proud'
Current noun 'other'
Current noun 'whole'
- Read 30000/30000 words so far


# 1. Determine approximately the top-10 words

In [6]:

#the statement already gives us the parameters used in the function and a hint
def add_to_reservoir(reservoir, item, max_reservoir_size):
    #If the reservoir is not full, add the new item
    if len(reservoir) < max_reservoir_size:
        reservoir.append(item)
    else:
        #Otherwise, we replace a random item in the reservoir with the new item
        pos = random.randint(0, max_reservoir_size - 1)
        reservoir[pos] = item
    assert len(reservoir) <= max_reservoir_size


In [7]:

#Again the statement gives us the parameters used in the function

def reservoir_sampling(filename, parts_of_speech, reservoir_size, max_words=-1, report_every=-1):
    reservoir = []
    words_read = 0  
    # we iterate through the words produced by read_by_parts_of_speech
    for word in read_by_parts_of_speech(filename, parts_of_speech,
                                        max_words=max_words,
                                        report_every=report_every):
        words_read += 1
        # If the reservoir is not full, add the new word
        if len(reservoir) < reservoir_size:
            
            reservoir.append(word)
        else:
            #otherwise, we decide whether to include the new word in the reservoir
            #we compute the probability of including the new word   
            p = reservoir_size / (words_read + 1)
            #we decide randomly whether to include the new word or not
            if random.random() < p:
               add_to_reservoir(reservoir, word, reservoir_size)
    #we return the number of words read and the reservoir
    return (words_read, reservoir)


In [8]:
# LEAVE THIS CODE AS-IS

reservoir_size = 1500
(items_seen, reservoir) = reservoir_sampling(INPUT_FILE, [POS_VERB], reservoir_size, max_words=MAX_WORDS)

print("Number of items seen    : %d" % items_seen)
print("Number of items sampled : %d" % len(reservoir) )

- Read 6000/30000 words so far
- Read 12000/30000 words so far
- Read 18000/30000 words so far
- Read 24000/30000 words so far
- Read 30000/30000 words so far
Number of items seen    : 30001
Number of items sampled : 1500


In [9]:
# Leave this code as-is

freq = {}
for item in reservoir:
    freq[item] = reservoir.count(item)

most_frequent_items = sorted([(frequency, word) for word, frequency in freq.items()], reverse=True)[:10]
for absolute_frequency, word in most_frequent_items:
    print("%d %s" % (absolute_frequency, word))

126 be
77 have
69 know
66 get
64 do
61 go
36 see
36 let
33 tell
32 make


In [10]:
# Code to the top 30 most frequent items with relative frequencies
#we start by computing the counts
freq = {}
for item in reservoir:
    if item in freq:
        freq[item] += 1
    else:
        freq[item] = 1

   
#we compute the total number of items in the reservoir
total_items = len(reservoir)

# We get the  top 30 most frequent
top30 = sorted([(count, word) for word, count in freq.items()],reverse=True)[:30]

# And finally we print results with percentages
print("Top 30 most frequent items with relative frequencies:\n")
for count, word in top30:
    percentage = (count / total_items) * 100
    print(f"{count:4d}  {percentage:6.2f}%  {word}")


Top 30 most frequent items with relative frequencies:

 126    8.40%  be
  77    5.13%  have
  69    4.60%  know
  66    4.40%  get
  64    4.27%  do
  61    4.07%  go
  36    2.40%  see
  36    2.40%  let
  33    2.20%  tell
  32    2.13%  make
  28    1.87%  think
  26    1.73%  take
  24    1.60%  say
  21    1.40%  come
  20    1.33%  believe
  18    1.20%  want
  18    1.20%  give
  15    1.00%  like
  14    0.93%  look
  14    0.93%  find
  13    0.87%  ask
  12    0.80%  talk
  12    0.80%  pay
  12    0.80%  leave
  12    0.80%  call
  11    0.73%  try
  11    0.73%  stop
  11    0.73%  put
  11    0.73%  keep
  10    0.67%  mind


In [ ]:
#We now run experiments with different reservoir sizes
MAX_WORDS = 10000

reservoir_sizes = [50, 100, 500, 1000, 5000]

print("EXPERIMENT WITH MAX_WORDS =", MAX_WORDS)
print()
#for each reservoir size, we perform reservoir sampling and compute estimates
for R in reservoir_sizes:
    print(f"Reservoir size: {R}")
    reservoir = []
    seen = 0
    # We perform reservoir sampling for verbs
    for w in read_by_parts_of_speech(INPUT_FILE, [POS_VERB], max_words=MAX_WORDS):
        seen += 1

        # indicator for progress (not rlly necessary)
        if seen % 10000 == 0:
            print(f" {seen}/{MAX_WORDS} words processed...")
        # Reservoir sampling logic
        if len(reservoir) < R:
            reservoir.append(w)
        else:
            p = R / seen
            if random.random() < p:
                reservoir[random.randint(0, R - 1)] = w
    # Now we compute frequencies in the reservoir
    freq = {}
    for w in reservoir:
        freq[w] = freq.get(w, 0) + 1
    # We get the top-5 words in the reservoir
    top5 = sorted(freq.items(), key=lambda x: x[1], reverse=True)[:5]

    print(f"\nWords read: {seen}")
    print("Top-5 words:\n")
    # For each of the top-5 words, we compute absolute and relative estimates
    for word, c in top5:
        est_abs = c * seen / R
        est_rel = est_abs / seen * 100
        print(f"{word:15s} | reservoir: {c:3d} | abs est: {est_abs:10.1f} | rel: {est_rel:6.3f}%")

    print()

EXPERIMENT WITH MAX_WORDS = 10000

Reservoir size: 50
- Read 2000/10000 words so far
- Read 4000/10000 words so far
- Read 6000/10000 words so far
- Read 8000/10000 words so far
- Read 10000/10000 words so far
 10000/10000 words processed...

Words read: 10005
Top-5 words:

be              | reservoir:   6 | abs est:     1200.6 | rel: 12.000%
tell            | reservoir:   3 | abs est:      600.3 | rel:  6.000%
have            | reservoir:   3 | abs est:      600.3 | rel:  6.000%
know            | reservoir:   2 | abs est:      400.2 | rel:  4.000%
get             | reservoir:   2 | abs est:      400.2 | rel:  4.000%

Reservoir size: 100
- Read 2000/10000 words so far
- Read 4000/10000 words so far
- Read 6000/10000 words so far
- Read 8000/10000 words so far
- Read 10000/10000 words so far
 10000/10000 words processed...

Words read: 10005
Top-5 words:

be              | reservoir:  10 | abs est:     1000.5 | rel: 10.000%
get             | reservoir:   8 | abs est:      800.4 | rel:  

From the experiments, it’s clear that the reservoir size strongly affects the accuracy of the estimated word frequencies. With very small reservoirs (50–100), the results were unstable and the most frequent words changed a lot, which shows that the sample was too small to represent the full dataset. As the reservoir size increased (500–1000), the estimates became more consistent and the top words appeared reliably. The largest reservoir (5000) produced the most stable and realistic frequencies, closely matching what we would expect from the full data(although it is also the most expensive in memory.)

If I had to choose a practical size, I would recommend around 1000, since it provides good accuracy without being too heavy. But if memory is not a concern, a larger reservoir like 5000 gives the best approximation.

# 2. Determine approximately the distinct number of words

In [12]:
# LEAVE THIS CODE AS-IS

def count_trailing_zeroes(number):
    count = 0
    while number & 1 == 0:
        count += 1
        number = number >> 1
    return count

In [13]:
# LEAVE THIS CODE AS-IS

def random_hash_function():
    # We use a cryptographically safe generator for the salt of our hash function
    salt = secrets.token_bytes(32)
    return lambda string: hash(string + str(salt))

In [ ]:
MAX_WORDS = 5000 #I decided to change maxwords becouse otherwise it took too long with 10 passes (the next cell was > 10 hours)
number_of_passes = 10
estimates = []
# We perform multiple passes to get multiple estimates
for i in range(number_of_passes):
    # For each pass, we create a new random hash function
    hash_function = random_hash_function()
    R = 0
    # We read the words and compute the maximum number of trailing zeroes
    for word in read_by_parts_of_speech(INPUT_FILE, [POS_VERB], max_words=MAX_WORDS):
        h = abs(hash_function(word))
        if h == 0:
            r = 0
        else:
            r = count_trailing_zeroes(h)
        if r > R:
            R = r
    # We compute the estimate based on the maximum number of trailing zeroes
    estimate = 2 ** R
    #And we store the estimate
    estimates.append(estimate)
    print("Estimate on pass %d: %d distinct words" % (i + 1, estimate))

- Read 1000/5000 words so far
- Read 2000/5000 words so far
- Read 3000/5000 words so far
- Read 4000/5000 words so far
- Read 5000/5000 words so far
Estimate on pass 1: 2048 distinct words
- Read 1000/5000 words so far
- Read 2000/5000 words so far
- Read 3000/5000 words so far
- Read 4000/5000 words so far
- Read 5000/5000 words so far
Estimate on pass 2: 16384 distinct words
- Read 1000/5000 words so far
- Read 2000/5000 words so far
- Read 3000/5000 words so far
- Read 4000/5000 words so far
- Read 5000/5000 words so far
Estimate on pass 3: 64 distinct words
- Read 1000/5000 words so far
- Read 2000/5000 words so far
- Read 3000/5000 words so far
- Read 4000/5000 words so far
- Read 5000/5000 words so far
Estimate on pass 4: 1024 distinct words
- Read 1000/5000 words so far
- Read 2000/5000 words so far
- Read 3000/5000 words so far
- Read 4000/5000 words so far
- Read 5000/5000 words so far
Estimate on pass 5: 1024 distinct words
- Read 1000/5000 words so far
- Read 2000/5000 word

In [16]:
# Leave this code as-is

print("* Average of estimates: %.1f" % statistics.mean(estimates))
print("* Median  of estimates: %.1f" % statistics.median(estimates))

* Average of estimates: 6099.2
* Median  of estimates: 1536.0


In [ ]:

#with max_words = 5000 this cell took around 424 minutes to run
number_of_runs = 3
passes_per_run = 10

#We will store the results for each part of speech here
POS_TOTAL = {"Nouns": [POS_NOUN], "Adjectives": [POS_ADJECTIVE], "Verbs": [POS_VERB]}

results = {}
#We perform the experiments for each part of speech
for pos_name, pos_list in POS_TOTAL.items():
    run_averages = []
    #We perform multiple runs to get more reliable estimates
    for run in range(number_of_runs):
        estimates = []
        #For each run, we perform multiple passes
        for p in range(passes_per_run):
            h = random_hash_function()
            R = 0
            # We read the words and compute the maximum number of trailing zeroes
            for word in read_by_parts_of_speech(INPUT_FILE, pos_list, max_words=-1, report_every=-1):
                hv = h(word)
                tz = count_trailing_zeroes(abs(hv))
                R = max(R, tz)
            # We compute the estimate based on the maximum number of trailing zeroes
            estimate = 2 ** R
            estimates.append(estimate)
            print(f"{pos_name} — Run {run+1}, Pass {p+1}: Estimate = {estimate}")
        # We compute the average estimate for this run
        avg_estimate = sum(estimates) / len(estimates)
        run_averages.append(avg_estimate)
        print(f"{pos_name} — Run {run+1}: Average estimate over 10 passes = {avg_estimate}")
    # We compute the median of the average estimates across runs
    median_estimate = statistics.median(run_averages)
    results[pos_name] = median_estimate
    print(f"{pos_name}: Median of average estimates across 3 runs = {median_estimate}\n")


Nouns — Run 1, Pass 1: Estimate = 16384
Nouns — Run 1, Pass 2: Estimate = 8192
Nouns — Run 1, Pass 3: Estimate = 32768
Nouns — Run 1, Pass 4: Estimate = 32768
Nouns — Run 1, Pass 5: Estimate = 32768
Nouns — Run 1, Pass 6: Estimate = 32768
Nouns — Run 1, Pass 7: Estimate = 8192
Nouns — Run 1, Pass 8: Estimate = 16384
Nouns — Run 1, Pass 9: Estimate = 4096
Nouns — Run 1, Pass 10: Estimate = 16384
Nouns — Run 1: Average estimate over 10 passes = 20070.4
Nouns — Run 2, Pass 1: Estimate = 131072
Nouns — Run 2, Pass 2: Estimate = 65536
Nouns — Run 2, Pass 3: Estimate = 32768
Nouns — Run 2, Pass 4: Estimate = 8192
Nouns — Run 2, Pass 5: Estimate = 16384
Nouns — Run 2, Pass 6: Estimate = 4096
Nouns — Run 2, Pass 7: Estimate = 32768
Nouns — Run 2, Pass 8: Estimate = 1048576
Nouns — Run 2, Pass 9: Estimate = 32768
Nouns — Run 2, Pass 10: Estimate = 32768
Nouns — Run 2: Average estimate over 10 passes = 140492.8
Nouns — Run 3, Pass 1: Estimate = 16384
Nouns — Run 3, Pass 2: Estimate = 32768
Nouns

<font size="+2" color="#003300">I hereby declare that I completed this practice myself, that my answers were not written by an AI-enabled code assistant, and that except for the code provided by the course instructors, all of my code, report, and figures were produced by myself.</font>